# Converts the JHU Cumulative Case Count Data to Daily Case Count Data

# TO DO:

- [X] Write the code that lets you convert the US data to long
- [X] Write the code that converts the column names in the Oxford data set to match the column names in the JHU dat
- [ ] Write the code that merges the countries to their offical alpha 3 code in the JHU: **[Link to Codes](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes)**
- [ ] **ALTERNATIVE TO ABOVE** use python-Levenshtein [Docs](https://rawgit.com/ztane/python-Levenshtein/master/docs/Levenshtein.html) distance to match similar country names 
> would still need to pair the high probability matches 
- [ ] Need to explore the missingness of the Oxford data. Sort the countries by GDP and examine what the missingness matrix looks like. **If you could run imputation on this data then you would have a major leg up on the other paper working on the similar topic. (on to of the other benefits to your paper)**
- [ ] Write the code that merges in the time series data for the diffent control measures
- [ ] Write the code transposes the combined data with the control measures included
- [ ] Write the code merges in the Country Safety Index data



In [1]:
from fuzzywuzzy import fuzz,process

In [2]:
from Levenshtein import distance,ratio

In [9]:
dir(process.extractOne())

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [23]:
lists = ['apdpled Juiceaaaaasfgdgfgfgfaa', 'penis', 'appleton']

In [24]:
process.extractOne('apples',lists)

('apdpled Juiceaaaaasfgdgfgfgfaa', 75)

In [16]:
ratio('there','the')

0.75

In [25]:
import os
import pandas as pd
c_measures_path = r'../../../covid-policy-tracker/data/timeseries'

In [63]:
school_close = pd.read_csv(f'{c_measures_path}/c1_schoolclosing.csv')

In [93]:
school_close.iloc[:184, 0:1]

,Unnamed: 0
0,Aruba
1,Afghanistan
2,Angola
3,Anguilla
4,Albania
...,...
179,Vietnam
180,Vanuatu
181,Yemen
182,South Africa


In [54]:
def Correcting_Col_Names(infile_path, dset):
    infile = pd.read_csv(infile_path)
    # og_col_list = infile.columns.to_list()
    df = pd.DataFrame(infile.columns.to_list())
    df['col'] = df[2:].apply(lambda x: pd.to_datetime(x).dt.strftime('X%m/X%d/%Y').str.replace('X0','').str.replace('X',''))
    df['col'][0] = 'country'
    df['col'][1] = 'country_code'
    cols = df.col.to_list()
    control = pd.read_csv(infile_path, names = cols, skiprows={0:1})
    control['var'] = dset
    indexNames = control[ control['country_code'].isna()].index
    control.drop(indexNames, inplace=True)
    return control

## Importing the Oxford Control Measures Data Set (Ordinal)

In [55]:
school = Correcting_Col_Names(f'{c_measures_path}/c1_schoolclosing.csv', 'school_close')
work = Correcting_Col_Names(f'{c_measures_path}/c2_workplaceclosing.csv', 'work_close')
pub_events = Correcting_Col_Names(f'{c_measures_path}/c3_cancelpublicevents.csv', 'public_events')
gatherings = Correcting_Col_Names(f'{c_measures_path}/c4_restrictionsongatherings.csv', 'large_gather')
pub_transpo = Correcting_Col_Names(f'{c_measures_path}/c5_closepublictransport.csv', 'public_transpo')
stay_home = Correcting_Col_Names(f'{c_measures_path}/c6_stayathomerequirements.csv' ,'stay_home')
domestic_travel = Correcting_Col_Names(f'{c_measures_path}/c7_domestictravel.csv' ,'domestic_travel')
int_travel = Correcting_Col_Names(f'{c_measures_path}/c8_internationaltravel.csv' ,'internat_travel')

In [60]:
import pandas as pd
import os
import sys
data_path = r'..\csse_covid_19_data\csse_covid_19_time_series'
out_data_path = r'..\Modified Data Sets'

case_pre = pd.read_csv(f'{data_path}/time_series_covid19_confirmed_global.csv')
death_pre = pd.read_csv(f'{data_path}/time_series_covid19_deaths_global.csv')

# case_pre_us = pd.read_csv(f'{data_path}/time_series_covid19_confirmed_US.csv')
# death_pre_us = pd.read_csv(f'{data_path}/time_series_covid19_deaths_US.csv')

import shutil
file_list = os.listdir(out_data_path)
for files in file_list:
    if files.find('.xlsx') >= 0:
        shutil.move(f'{out_data_path}/{files}',f'{out_data_path}/ARCHIVE/{files}')
def DF_Transform(df, outcome):
    global data_path
    global out_data_path
    df.drop(labels={'Lat','Long'},axis=1, inplace = True)
    df = df.groupby(by='Country/Region').sum().T.apply(lambda x: x-x.shift(1),axis=0)
    df.rename(columns={'Country/Region':'Date'},inplace=True)
    df.columns = df.columns.str.replace(' ','_').str.lower()
    df = df.reset_index()
    df.rename(columns={'index':'date'},inplace= True)
    df.date = pd.to_datetime(df.date).dt.date
    df['var'] = outcome[16:]
    filename = f'{out_data_path}/{outcome} (Through {df.date.max()}).xlsx'
    df.to_excel(filename, index=False)
    return df
case = DF_Transform(case_pre, 'Global COVID-19 Case Count')
death = DF_Transform(death_pre, 'Global COVID-19 Death Count')
# case = DF_Transform(case_pre_us, 'US COVID-19 Case Count')
# death = DF_Transform(death_pre_us, 'US COVID-19 Death Count')

In [36]:
case

Country/Region,date,afghanistan,albania,algeria,andorra,angola,antigua_and_barbuda,argentina,armenia,australia,...,uruguay,uzbekistan,venezuela,vietnam,west_bank_and_gaza,western_sahara,yemen,zambia,zimbabwe,var
0,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Case Count
1,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,Case Count
2,2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Case Count
3,2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Case Count
4,2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Case Count
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2020-09-16,40.0,144.0,232.0,45.0,106.0,0.0,11674.0,257.0,35.0,...,29.0,586.0,761.0,0.0,756.0,0.0,3.0,68.0,22.0,Case Count
239,2020-09-17,17.0,132.0,228.0,0.0,114.0,0.0,12701.0,295.0,48.0,...,20.0,612.0,868.0,3.0,837.0,0.0,3.0,41.0,35.0,Case Count
240,2020-09-18,11.0,125.0,219.0,81.0,59.0,0.0,11945.0,239.0,24.0,...,14.0,626.0,890.0,2.0,558.0,0.0,2.0,94.0,14.0,Case Count
241,2020-09-19,36.0,153.0,210.0,0.0,53.0,1.0,9276.0,244.0,13.0,...,14.0,739.0,775.0,0.0,602.0,0.0,2.0,48.0,25.0,Case Count


In [31]:
case.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Columns: 189 entries, date to zimbabwe
dtypes: float64(188), object(1)
memory usage: 358.9+ KB


In [ ]:
case.describe()

## trying to work out country linking stuff

In [113]:
jhu_country = case_pre['Country/Region']
ox_country = school_close['Unnamed: 0'].dropna()

In [97]:
jhu_country.shape

(266,)

In [99]:
jhu_country

0             Afghanistan
1                 Albania
2                 Algeria
3                 Andorra
4                  Angola
              ...        
261    West Bank and Gaza
262        Western Sahara
263                 Yemen
264                Zambia
265              Zimbabwe
Name: Country/Region, Length: 266, dtype: object

In [119]:
for countries in ox_country:
    results = process.extractOne(countries,list(jhu_country))
    if results[1] < 100:
        print(results, countries)

('Barbados', 72) Aruba
('Angola', 71) Anguilla
('Peru', 68) Bermuda
('Congo (Brazzaville)', 86) Democratic Republic of Congo
('Congo (Brazzaville)', 90) Congo
('Cabo Verde', 80) Cape Verde
('Oman', 68) Cayman Islands
('Central African Republic', 86) Czech Republic
('Finland', 64) Falkland Islands
('Malta', 72) Gibraltar
('Grenada', 75) Greenland
('Nicaragua', 77) Guam
('Congo (Kinshasa)', 57) Hong Kong
('Central African Republic', 86) Kyrgyz Republic
('Korea, South', 95) South Korea
('Monaco', 73) Macao
('Guyana', 62) Myanmar
('Sierra Leone', 61) Montserrat
('Iran', 68) Pitcairn Islands
('Peru', 68) Puerto Rico
('Eswatini', 59) Palestine
('Oman', 68) Solomon Islands
('Central African Republic', 86) Slovak Republic
('Finland', 64) Turks and Caicos Islands
('Turkey', 75) Turkmenistan
('United Arab Emirates', 86) United States
('Finland', 64) British Virgin Islands
('Antigua and Barbuda', 51) Vanuatu


TypeError: expected string or bytes-like object

In [111]:
ox_country.

,Unnamed: 0
0,Aruba
1,Afghanistan
2,Angola
3,Anguilla
4,Albania
...,...
179,Vietnam
180,Vanuatu
181,Yemen
182,South Africa
